https://machinelearningmastery.com/building-a-multiclass-classification-model-in-pytorch/

In [3]:
import pandas as pd
data = pd.read_csv("D:\Projects\VsCode\Python\img_processing_system\practice\pytorch\data\iris.csv", header=None)
X = data.iloc[:, 0:4]
print(X)
y = data.iloc[:, 4:]
print(y)

       0    1    2    3
0    5.1  3.5  1.4  0.2
1    4.9  3.0  1.4  0.2
2    4.7  3.2  1.3  0.2
3    4.6  3.1  1.5  0.2
4    5.0  3.6  1.4  0.2
..   ...  ...  ...  ...
145  6.7  3.0  5.2  2.3
146  6.3  2.5  5.0  1.9
147  6.5  3.0  5.2  2.0
148  6.2  3.4  5.4  2.3
149  5.9  3.0  5.1  1.8

[150 rows x 4 columns]
                  4
0       Iris-setosa
1       Iris-setosa
2       Iris-setosa
3       Iris-setosa
4       Iris-setosa
..              ...
145  Iris-virginica
146  Iris-virginica
147  Iris-virginica
148  Iris-virginica
149  Iris-virginica

[150 rows x 1 columns]
